In [ ]:
from robomimic.config import config_factory
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.file_utils as FileUtils
import json
import os

config_file = "../config/bc_rnn.json"
ext_cfg = json.load(open(config_file, 'r'))
config = config_factory(ext_cfg["algo_name"])
# update config with external json - this will throw errors if
# the external config has keys not present in the base algo config
with config.unlocked():
    config.update(ext_cfg)
# get torch device
device = TorchUtils.get_torch_device(try_to_use_cuda=config.train.cuda)
config.train.data = "../../datasets/test/square_ph.hdf5"
# send output to a temporary directory
config.train.output_dir = "/tmp/tmp_trained_models"
# lock config to prevent further modifications and ensure missing keys raise errors
config.lock()

In [ ]:
# read config to set up metadata for observation modalities (e.g. detecting rgb observations)
ObsUtils.initialize_obs_utils_with_config(config)

print(ObsUtils.OBS_SHAPES)

# make sure the dataset exists
dataset_path = os.path.expanduser(config.train.data)
if not os.path.exists(dataset_path):
    raise Exception("Dataset at provided path {} not found!".format(dataset_path))

# # load basic metadata from training file
# print("\n============= Loaded Environment Metadata =============")
# env_meta = FileUtils.get_env_metadata_from_dataset(dataset_path=config.train.data)
# shape_meta = FileUtils.get_shape_metadata_from_dataset(
#     dataset_path=config.train.data,
#     all_obs_keys=config.all_obs_keys,
#     verbose=True
# )
# print(shape_meta["all_shapes"])
# print(config.observation.modalities)